In [7]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.ml.feature import Normalizer, StandardScaler
# import random
# import logging
import time
import json
from datetime import datetime
from pymongo import MongoClient

schema = StructType([ \
    StructField("RGIID",StringType(),True), 
    StructField("Time",StringType(),True), 
    StructField("Area",FloatType(),True), 
    StructField("dh",FloatType(),True), 
    StructField("err_dh",FloatType(),True),                  
  ])

client = MongoClient('localhost', 27017)
db = client.ICS5114
collection = db.glacial_collection

temp_collection = []

cursor = collection.find({})
for document in cursor:  
    try:
        temp_collection.append((document["rgiid"],document["time"],document["area"],document["dh"],document["err_dh"]))
    except Exception as e: print()

df = spark.createDataFrame(data=temp_collection,schema=schema)
df = df.withColumn("Time_formatted", f.to_date(f.col("Time"),"dd/MM/yyyy"))
df = df.sort("RGIID","Time_formatted")

df.printSchema()
df.show()

root
 |-- RGIID: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Area: float (nullable = true)
 |-- dh: float (nullable = true)
 |-- err_dh: float (nullable = true)
 |-- Time_formatted: date (nullable = true)

+--------------+----------+---------+------+------+--------------+
|         RGIID|      Time|     Area|    dh|err_dh|Time_formatted|
+--------------+----------+---------+------+------+--------------+
|RGI60-06.00001|01/01/2000|4903000.0|   0.0| 2.342|    2000-01-01|
|RGI60-06.00001|31/01/2000|4903000.0|-0.064|  2.32|    2000-01-31|
|RGI60-06.00001|02/03/2000|4903000.0|-0.143| 2.298|    2000-03-02|
|RGI60-06.00001|01/04/2000|4903000.0|-0.247| 2.275|    2000-04-01|
|RGI60-06.00001|02/05/2000|4903000.0|-0.412| 2.251|    2000-05-02|
|RGI60-06.00001|01/06/2000|4903000.0|-0.656| 2.227|    2000-06-01|
|RGI60-06.00001|02/07/2000|4903000.0|-0.931| 2.205|    2000-07-02|
|RGI60-06.00001|01/08/2000|4903000.0|-1.152| 2.196|    2000-08-01|
|RGI60-06.00001|01/09/2000|4903000.

In [6]:
df.describe().show()

+-------+--------------+----------+--------------------+-----------------+-----------------+
|summary|         RGIID|      Time|                Area|               dh|           err_dh|
+-------+--------------+----------+--------------------+-----------------+-----------------+
|  count|        136887|    136887|              136887|           136887|           136887|
|   mean|          null|      null|1.9471436196570896E7|-4.64186489664232|2.572899574157931|
| stddev|          null|      null|1.1690602907993484E8|7.298624478050708| 0.95197350234568|
|    min|RGI60-06.00001|01/01/2000|             44000.0|          -91.909|            0.413|
|    max|RGI60-06.00568|31/12/2016|        1.56121805E9|           34.833|            12.39|
+-------+--------------+----------+--------------------+-----------------+-----------------+



In [8]:
# count null values
import pyspark.sql.functions as f
nulls = df.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in df.columns])
nulls.show()

+-----+----+----+---+------+--------------+
|RGIID|Time|Area| dh|err_dh|Time_formatted|
+-----+----+----+---+------+--------------+
|    0|   0|   0|  0|     0|             0|
+-----+----+----+---+------+--------------+



In [9]:
# pearson correlation
from pyspark.sql.functions import * 
print("Pearrson correlation between Area and Glacier elavation change")
df.stat.corr("Area", "dh")

Pearrson correlation between Area and Glacier elavation change


-0.1457903292574538

In [10]:
df.columns

['RGIID', 'Time', 'Area', 'dh', 'err_dh', 'Time_formatted']

In [14]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

feat_cols = ["Time","Area","dh"]

vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')
final_data = vec_assembler.transform(df)

scaler = StandardScaler(inputCol = "features", outputCol="scaledFeatures", withStd=True, withMean=False )
scalerModel = scaler.fit(final_data)
c_final_data = scalerModel.transform(final_data)

from pyspark.ml.clustering import *

kmeans3 = KMeans(featuresCol="scaledFeatures", k=3)
kmeans2 = KMeans(featuresCol="scaledFeatures", k=2)

model_k3 = kmeans3.fit(c_final_data)
model_k2 = kmeans2.fit(c_final_data)

model_k3.transform(c_final_data).groupBy('prediction').count().show()

IllegalArgumentException: Data type string of column Time is not supported.

In [18]:
print("Number of Distinct Glacials:")
df.select('RGIID').distinct().count()

Number of Distinct Glacials:


568

In [17]:
new_df = df.sort('Time_formatted')

In [19]:
from pyspark.sql.window import Window

window_days = 3
window = (
    Window
    .partitionBy(f.col("RGIID"))
    .orderBy(f.col("Time_formatted").cast("timestamp").cast("long"))
    .rowsBetween(-window_days, Window.currentRow-1)
)
new_all_data = (df
    .withColumn("sum",f.sum(f.col("dh")).over(window))
    .withColumn("mean",f.avg(f.col("dh")).over(window))
    .withColumn("min",f.min(f.col("dh")).over(window))
    .withColumn("max",f.max(f.col("dh")).over(window)))
#     .withColumn("stddev",f.stddev(f.col("dh")).over(window)))

w = (
    Window
    .partitionBy(f.col("RGIID"))
    .orderBy(f.col("Time_formatted").cast("timestamp").cast("long"))
) 

# creating difference between last known dh and first known dh value in rolling window.
df_lagger = new_all_data.withColumn('diff', f.lag(f.col("dh"),1).over(w) - f.lag(f.col("dh"),window_days).over(w))
# df_lagger.show()

# now including sign 
df_including_sign = df_lagger.withColumn("Trend", f.signum(f.col("diff")))
df_including_sign.show()


+--------------+----------+---------+------+------+--------------+--------------------+--------------------+------+------+------------+-----+
|         RGIID|      Time|     Area|    dh|err_dh|Time_formatted|                 sum|                mean|   min|   max|        diff|Trend|
+--------------+----------+---------+------+------+--------------+--------------------+--------------------+------+------+------------+-----+
|RGI60-06.00001|01/01/2000|4903000.0|   0.0| 2.342|    2000-01-01|                null|                null|  null|  null|        null| null|
|RGI60-06.00001|31/01/2000|4903000.0|-0.064|  2.32|    2000-01-31|                 0.0|                 0.0|   0.0|   0.0|        null| null|
|RGI60-06.00001|02/03/2000|4903000.0|-0.143| 2.298|    2000-03-02|-0.06400000303983688|-0.03200000151991844|-0.064|   0.0|        null| null|
|RGI60-06.00001|01/04/2000|4903000.0|-0.247| 2.275|    2000-04-01| -0.2070000097155571|-0.06900000323851903|-0.143|   0.0|      -0.143| -1.0|
|RGI60

In [20]:
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

formatted_df = df_including_sign.withColumn('Year', f.substring(f.col('Time'), 7, 10).cast('int'))
    
# vec_assembler = VectorAssembler(inputCols = ["Area", "dh","sum", "min", "max", "mean"], outputCol='features')
vec_assembler = VectorAssembler(inputCols = ["Area","mean", "diff"], outputCol='features')

output_df = vec_assembler.transform(formatted_df.dropna())
output_df.printSchema()

output_df.show()

root
 |-- RGIID: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Area: float (nullable = true)
 |-- dh: float (nullable = true)
 |-- err_dh: float (nullable = true)
 |-- Time_formatted: date (nullable = true)
 |-- sum: double (nullable = true)
 |-- mean: double (nullable = true)
 |-- min: float (nullable = true)
 |-- max: float (nullable = true)
 |-- diff: float (nullable = true)
 |-- Trend: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- features: vector (nullable = true)

+--------------+----------+---------+------+------+--------------+--------------------+--------------------+------+------+------------+-----+----+--------------------+
|         RGIID|      Time|     Area|    dh|err_dh|Time_formatted|                 sum|                mean|   min|   max|        diff|Trend|Year|            features|
+--------------+----------+---------+------+------+--------------+--------------------+--------------------+------+------+------------+-----+----+--

In [21]:
final_data = output_df.select('features','Trend')
final_data = final_data.withColumn("Trend",f.when(f.col("Trend") <= 0, 0).otherwise(f.col("Trend")))
final_data.show()

+--------------------+-----+
|            features|Trend|
+--------------------+-----+
|[4903000.0,-0.069...|  0.0|
|[4903000.0,-0.151...|  0.0|
|[4903000.0,-0.267...|  0.0|
|[4903000.0,-0.438...|  0.0|
|[4903000.0,-0.666...|  0.0|
|[4903000.0,-0.912...|  0.0|
|[4903000.0,-1.117...|  0.0|
|[4903000.0,-1.238...|  0.0|
|[4903000.0,-1.279...|  0.0|
|[4903000.0,-1.279...|  1.0|
|[4903000.0,-1.279...|  0.0|
|[4903000.0,-1.304...|  0.0|
|[4903000.0,-1.355...|  0.0|
|[4903000.0,-1.428...|  0.0|
|[4903000.0,-1.535...|  0.0|
|[4903000.0,-1.695...|  0.0|
|[4903000.0,-1.912...|  0.0|
|[4903000.0,-2.147...|  0.0|
|[4903000.0,-2.341...|  0.0|
|[4903000.0,-2.450...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [23]:
from pyspark.sql import Row

train_df = final_data.where((f.col('Year') != '2020') & (col('Trend').isNotNull()))
test_df = final_data.where((f.col('Year') == '2020') & (col('Trend').isNotNull()))

print("Training data")
train_df.show()
print("----------------------------")
print("Testing data")
test_df.show()

Training data
+--------------------+-----+
|            features|Trend|
+--------------------+-----+
|[4903000.0,-0.069...|  0.0|
|[4903000.0,-0.151...|  0.0|
|[4903000.0,-0.267...|  0.0|
|[4903000.0,-0.438...|  0.0|
|[4903000.0,-0.666...|  0.0|
|[4903000.0,-0.912...|  0.0|
|[4903000.0,-1.117...|  0.0|
|[4903000.0,-1.238...|  0.0|
|[4903000.0,-1.279...|  0.0|
|[4903000.0,-1.279...|  1.0|
|[4903000.0,-1.279...|  0.0|
|[4903000.0,-1.304...|  0.0|
|[4903000.0,-1.355...|  0.0|
|[4903000.0,-1.428...|  0.0|
|[4903000.0,-1.535...|  0.0|
|[4903000.0,-1.695...|  0.0|
|[4903000.0,-1.912...|  0.0|
|[4903000.0,-2.147...|  0.0|
|[4903000.0,-2.341...|  0.0|
|[4903000.0,-2.450...|  0.0|
+--------------------+-----+
only showing top 20 rows

----------------------------
Testing data
+--------------------+-----+
|            features|Trend|
+--------------------+-----+
|[4903000.0,-6.634...|  1.0|
|[67000.0,1.772000...|  1.0|
|[2712000.0,-10.75...|  1.0|
|[448000.0,-4.6656...|  0.0|
|[77000.0,-1.45099.

In [24]:
from pyspark.ml.classification import *
LR_model = LogisticRegression(labelCol = 'Trend')
LR_model = LR_model.fit(train_df)
predictions = LR_model.evaluate(test_df)

predictions = predictions.predictions
predictions.show()

C:\spark\python\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+-----+--------------------+--------------------+----------+
|            features|Trend|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[4903000.0,-6.634...|  1.0|[-8352.5898663134...|           [0.0,1.0]|       1.0|
|[67000.0,1.772000...|  1.0|[-2024.2197390051...|           [0.0,1.0]|       1.0|
|[2712000.0,-10.75...|  1.0|[-4667.4723631592...|           [0.0,1.0]|       1.0|
|[448000.0,-4.6656...|  0.0|[1830.51994698684...|           [1.0,0.0]|       0.0|
|[77000.0,-1.45099...|  0.0|[1660.93923337743...|           [1.0,0.0]|       0.0|
|[119000.0,-7.2006...|  1.0|[-2873.1855691670...|           [0.0,1.0]|       1.0|
|[165000.0,-4.5836...|  1.0|[-7770.6428866989...|           [0.0,1.0]|       1.0|
|[86000.0,0.340666...|  0.0|[1491.29885444625...|           [1.0,0.0]|       0.0|
|[6608000.0,-8.237...|  1.0|[-3067.1451708554...|           [0.0,1.0]|       1.0|
|[277000.0,-2.04

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# evaluating predictions
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol="Trend")
evaluator.evaluate(predictions)

1.0

In [ ]:
# https://www.youtube.com/watch?v=oDTJxEl95Go  - used as guide for spark.ml library

In [26]:
from pyspark.ml.classification import LinearSVC

svm = LinearSVC(maxIter=10, regParam=0.1 ,labelCol = 'Trend')
SVM_model = svm.fit(train_df)

svm_predictions = SVM_model.evaluate(test_df)
svm_predictions = svm_predictions.predictions
svm_predictions.show()

+--------------------+-----+--------------------+----------+
|            features|Trend|       rawPrediction|prediction|
+--------------------+-----+--------------------+----------+
|[4903000.0,-6.634...|  1.0|[-1.0030995043271...|       1.0|
|[67000.0,1.772000...|  1.0|[-0.2574118189273...|       1.0|
|[2712000.0,-10.75...|  1.0|[-0.4241631559555...|       1.0|
|[448000.0,-4.6656...|  0.0|[0.38219824201411...|       0.0|
|[77000.0,-1.45099...|  0.0|[0.30701235046642...|       0.0|
|[119000.0,-7.2006...|  1.0|[-0.2314579126406...|       1.0|
|[165000.0,-4.5836...|  1.0|[-0.9548093217350...|       1.0|
|[86000.0,0.340666...|  0.0|[0.25464478883786...|       0.0|
|[6608000.0,-8.237...|  1.0|[-0.2421486505337...|       1.0|
|[277000.0,-2.0480...|  0.0|[0.09059244811404...|       0.0|
|[167000.0,3.46999...|  1.0|[-0.0114574573179...|       1.0|
|[128000.0,-11.118...|  1.0|[-0.1449565637031...|       1.0|
|[101000.0,-1.5883...|  1.0|[-1.8225191705240...|       1.0|
|[164000.0,-0.4633...|  

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
svm_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol="Trend")
svm_evaluator.evaluate(svm_predictions)

0.9393203883495146

In [28]:
from pyspark.ml.regression import LinearRegression

formatted_df = df_including_sign.withColumn('Year', f.year(f.col('Time_formatted')))
formatted_df.show() 

+--------------+----------+---------+------+------+--------------+--------------------+--------------------+------+------+------------+-----+----+
|         RGIID|      Time|     Area|    dh|err_dh|Time_formatted|                 sum|                mean|   min|   max|        diff|Trend|Year|
+--------------+----------+---------+------+------+--------------+--------------------+--------------------+------+------+------------+-----+----+
|RGI60-06.00001|01/01/2000|4903000.0|   0.0| 2.342|    2000-01-01|                null|                null|  null|  null|        null| null|2000|
|RGI60-06.00001|31/01/2000|4903000.0|-0.064|  2.32|    2000-01-31|                 0.0|                 0.0|   0.0|   0.0|        null| null|2000|
|RGI60-06.00001|02/03/2000|4903000.0|-0.143| 2.298|    2000-03-02|-0.06400000303983688|-0.03200000151991844|-0.064|   0.0|        null| null|2000|
|RGI60-06.00001|01/04/2000|4903000.0|-0.247| 2.275|    2000-04-01| -0.2070000097155571|-0.06900000323851903|-0.143|   

In [32]:
print("Null Counter")

nulls = formatted_df.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in formatted_df.columns])
nulls.show()

Null Counter
+-----+----+----+---+------+--------------+---+----+---+---+----+-----+----+
|RGIID|Time|Area| dh|err_dh|Time_formatted|sum|mean|min|max|diff|Trend|Year|
+-----+----+----+---+------+--------------+---+----+---+---+----+-----+----+
|    0|   0|   0|  0|     0|             0|568| 568|568|568|1704| 1704|   0|
+-----+----+----+---+------+--------------+---+----+---+---+----+-----+----+



In [34]:
# vec_assembler = VectorAssembler(inputCols = ["Area", "dh","sum", "min", "max", "mean"], outputCol='features')
vec_assembler = VectorAssembler(inputCols = ["Area","mean"], outputCol='features')

# very important to drop any null values here as the feature vectors must have the same shape
output_df = vec_assembler.transform(formatted_df.dropna())
output_df.printSchema()

final_data = output_df.select('features','dh')
# final_data = final_data.withColumn("dh",f.when(f.col("Trend") <= 0, 0).otherwise(f.col("Trend")))

train_df = final_data.where(f.col('Year') != '2020')
test_df = final_data.where(f.col('Year') == '2020')

train_df = train_df.where((col('dh').isNotNull()))
test_df = test_df.where((col('dh').isNotNull()))

lm = LinearRegression(labelCol="dh")
LR_model = lm.fit(train_df)

LR_predictions_res = LR_model.evaluate(test_df)
LR_predictions = LR_predictions_res.predictions
LR_predictions.show()

root
 |-- RGIID: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Area: float (nullable = true)
 |-- dh: float (nullable = true)
 |-- err_dh: float (nullable = true)
 |-- Time_formatted: date (nullable = true)
 |-- sum: double (nullable = true)
 |-- mean: double (nullable = true)
 |-- min: float (nullable = true)
 |-- max: float (nullable = true)
 |-- diff: float (nullable = true)
 |-- Trend: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- features: vector (nullable = true)

+--------------------+-------+--------------------+
|            features|     dh|          prediction|
+--------------------+-------+--------------------+
|[4903000.0,-6.634...| -6.326|  -6.713959201897589|
|[67000.0,1.772000...|  1.904|  1.7382289042337273|
|[2712000.0,-10.75...|-10.577| -10.855493572890087|
|[448000.0,-4.6656...|  -4.67|  -4.733976758754863|
|[77000.0,-1.45099...| -1.519| -1.5020455518562374|
|[119000.0,-7.2006...| -7.132|  -7.282531219014964|
|[165000.0,-4.58

In [36]:
print("Linear Regression - Further Analysis")
print("RMSE -> ",LR_predictions_res.rootMeanSquaredError)
print("R2 -> ",LR_predictions_res.r2)
print("MSE -> ",LR_predictions_res.meanSquaredError)
print("MAE -> ",LR_predictions_res.meanAbsoluteError)

Linear Regression - Further Analysis
RMSE ->  0.4843833827312313
R2 ->  0.9976002182314097
MSE ->  0.23462726146615054
MAE ->  0.33434976223643426


In [ ]:
# i dub thee the christina cell

import pandas as pd

# time and mean --- need to find another way to deal with this maybe? we cant do this for each RGIID
pdf = (
    df_including_sign.where(f.col('RGIID') == 'RGI60-06.00001')
      .select(
        "Time_formatted",
        "mean",
    )
    .orderBy("Time_formatted")
    .toPandas()
)

pdf.plot.line(x="Time_formatted", y="mean")

# time and trend --- need to find another way to deal with this maybe? we cant do this for each RGIID
pdf = (
    df_including_sign.where(f.col('RGIID') == 'RGI60-06.00001')
      .select(
        "Time_formatted",
        "Trend",
    )
    .orderBy("Time_formatted")
    .toPandas()
)

pdf.plot.line(x="Time_formatted", y="Trend")

In [ ]:
# print(df_including_sign.select('RGIID').distinct().count())  # 568

# formatted_df.schema  # using formatted_df bc shema has year
# do i add matlotlib inline somwhere?

# group by year in order to get average dh
plot_test = formatted_df.groupBy('Year').agg(f.avg('dh').alias('avg_dh'))

pdf = (
    plot_test
      .select(
        "avg_dh",
        "Year",
    )
    .orderBy("Year")
    .toPandas()
)
pdf.plot.line(x="Year", y="avg_dh")


# group by year and RGIID in order to get average dh
plot_test = formatted_df.groupBy('Year', 'RGIID').agg(f.avg('dh').alias('avg_dh'))

pdf = (
    plot_test.where(f.col('RGIID') == 'RGI60-06.00001')
      .select(
        "avg_dh",
        "Year",
    )
    .orderBy("Year")
    .toPandas()
)

pdf.plot.line(x="Year", y="avg_dh")

In [ ]:
# THE ABOVE, BUT AT MONTH LEVEL

formatted_df_month = df_including_sign.withColumn('YearMonth', f.concat(f.substring(f.col('Time'), 7, 4), #)) 
                                                                  f.substring(f.col('Time'), 4, 2)).cast('int'))
# formatted_df_month.show()
# WEIRD VALUES ARE SHOWING UP BC THEY'RE TOO TIGHT - CHECK THE DATAFRAME AND THERE'S SO STUPID VALUES LIKE 201250

# group by year in order to get average dh
plot_test = formatted_df_month.groupBy('YearMonth').agg(f.avg('dh').alias('avg_dh'))

pdf = (
    plot_test
      .select(
        "avg_dh",
        "YearMonth",
    )
    .orderBy("YearMonth")
    .toPandas()
)
pdf.plot.line(x="YearMonth", y="avg_dh")


# group by year and RGIID in order to get average dh
plot_test = formatted_df_month.groupBy('YearMonth', 'RGIID').agg(f.avg('dh').alias('avg_dh'))

pdf = (
    plot_test.where(f.col('RGIID') == 'RGI60-06.00001')
      .select(
        "avg_dh",
        "YearMonth",
    )
    .orderBy("YearMonth")
    .toPandas()
)

pdf.plot.line(x="YearMonth", y="avg_dh")

In [ ]:
# import sys
# !{sys.executable} -m pip install seaborn

# import sys
# !{sys.executable} -m pip install mylib

In [ ]:
# df_including_sign.schema
# df_including_sign.show()

import seaborn as sns  # ikollna bzonn infaqqghu pip install imma tidher kif hemm go snipping tool

plot_test = formatted_df.groupBy('Year', 'RGIID', 'Trend').count()
plot_test = plot_test.withColumn('Year', f.col('Year').cast('string'))


plot_test.show()

# run as is and try running with .where()
pdf = (
    plot_test.where(f.col('RGIID') == 'RGI60-06.00001')
      .select(
#         "RGIID",
        "Year",
        "Trend",
        "count"
    )
    .orderBy("Year")
#     .toPandas()
)

# pdf.show()

# sns.histplot(x='Year',hue='Trend',data=pdf ,multiple="dodge")


In [ ]:
print("jesus")

In [ ]:
pdf.hist(x="Year")